## Introduction: Business Problem <a name="introduction"></a>

I want to start a new highend Bangladeshi fusion restaurant. I will open it in toronto area. I want to select a location based on following

1. If the neighborhood has any Bangladeshi or Bengali lauguage spoken people
2. Presense of other Bangladeshi or indian (cusine close to Bangladeshi) restaurants in the neighborhood

People who want to try new food in a highend environment will be very interested to my restaurant. Also investors and people who like different cultures and very interested to try diferent food and at the same time earn a good income will care my business analysis.


## Data and Solution idea:

In previous projects, we learned how to get the data from the website. Also How to use Foursquare app.


1. I will use Toronto data to get the selection information. My data source has a column named Second most common language (after English) by name. From that column, I will find locations name where Bengali lauguage are spoken. 
2. With those location names I will find the logitude and latitude unising geolocator.geocode
3. Number of Indian restaurants and location in every selected neighborhood will be obtained using Foursquare API


### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76256 sha256=e18dd19587497360bfce4ba69098df2d78183327a502cc56179d965072bbb5a7
  Stored in directory: c:\users\ferdous\appdata\local\pip\cache\wheels\ef\4c\4a\17fd3d7fb7b6243d5a7a8d165870cd5c6ad2ec4c0582f039e4
Successfully built folium
Folium installed
Libraries imported.


### Data source:
https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods 
this website contains the main data in three tables. Following codes are used to create a main dataframe named 'df' combining the three tables.

In [2]:
Total_table= pd.read_html('https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods')
df=Total_table[1]
df_2=Total_table[2]
df_3=Total_table[5]

In [3]:
df.append(df_2, ignore_index=True)

,Name,FM,Census Tracts,Population,Land area (km2),Density (people/km2),% Change in Population since 2001,Average Income,Transit Commuting %,% Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map
0,Crescent Town,EY,0190.01,8157,0.40,20393,-10.0,23021,24.5,20.3,Bengali (18.1%),18.1% Bengali,NaN
1,Governor's Bridge/Bennington Heights,EY,0186.00,2112,1.87,1129,4.0,129904,7.1,13.3,Polish (1.4%),01.4% Polish,NaN
2,Leaside,EY,"0195.00, 0196.00",13876,2.81,4938,3.0,82670,9.7,10.5,Bulgarian (0.4%),00.4% Bulgarian,NaN
3,O'Connor–Parkview,EY,"0189.00, 0190.02, 0191.00, 0192.00, 0193.00",17740,4.94,3591,-6.1,33517,15.8,19.4,Urdu (3.2%),03.2% Urdu,NaN
4,Old East York,EY,"0180.00, 0181.01, 0181.02, 0182.00, 0183.00, 0...",52220,7.94,6577,-4.6,33172,22.0,19.1,Greek (4.3%),04.3% Greek,NaN
5,Thorncliffe Park,EY,"0194.01, 0194.02, 0194.03, 0194.04",17949,3.09,5809,9.1,25340,16.7,32.5,Urdu (21.5%),21.5% Urdu,NaN
6,Alderwood,E,"0211.00, 0212.00",11656,4.94,2360,-4.0,35239,8.8,8.5,Polish (6.2%),06.2% Polish,NaN
7,Centennial,E,"0236.01, 0236.02",12565,4.94,2544,0.5,34867,11.5,8.8,Polish (2.7%),02.7% Polish,NaN
8,Clairville,E,"0248.03, 0249.03",8506,6.71,1268,-3.3,26610,13.2,7.2,Punjabi (12.0%),12.0% Punjabi,NaN
9,Eatonville,E,"0213.00, 0221.02, 0222.01, 0222.02",19131,11.26,1699,4.3,36206,12.6,13.4,Serbian (3.2%),03.2% Serbian,NaN


In [4]:
df=df.append(df_3, ignore_index=True)

#### Getting the column names

In [5]:
df.columns
 

Index(['Name', 'FM', 'Census Tracts', 'Population', 'Land area (km2)',
       'Density (people/km2)', '% Change in Population since 2001',
       'Average Income', 'Transit Commuting %', '% Renters',
       'Second most common language (after English) by name',
       'Second most common language (after English) by percentage', 'Map'],
      dtype='object')

#### Dropping the nan values 

In [6]:
df = df[df['Second most common language (after English) by name'].notna()]
df = df.reset_index(drop=True)

In [7]:
df.shape

(157, 13)

#### Resetting the index after dropping nan

In [8]:
df.reset_index()

,index,Name,FM,Census Tracts,Population,Land area (km2),Density (people/km2),% Change in Population since 2001,Average Income,Transit Commuting %,% Renters,Second most common language (after English) by name,Second most common language (after English) by percentage,Map
0,0,Crescent Town,EY,0190.01,8157.0,0.40,20393.0,-10.0,23021.0,24.5,20.3,Bengali (18.1%),18.1% Bengali,NaN
1,1,Governor's Bridge/Bennington Heights,EY,0186.00,2112.0,1.87,1129.0,4.0,129904.0,7.1,13.3,Polish (1.4%),01.4% Polish,NaN
2,2,Leaside,EY,"0195.00, 0196.00",13876.0,2.81,4938.0,3.0,82670.0,9.7,10.5,Bulgarian (0.4%),00.4% Bulgarian,NaN
3,3,O'Connor–Parkview,EY,"0189.00, 0190.02, 0191.00, 0192.00, 0193.00",17740.0,4.94,3591.0,-6.1,33517.0,15.8,19.4,Urdu (3.2%),03.2% Urdu,NaN
4,4,Old East York,EY,"0180.00, 0181.01, 0181.02, 0182.00, 0183.00, 0...",52220.0,7.94,6577.0,-4.6,33172.0,22.0,19.1,Greek (4.3%),04.3% Greek,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,152,Woburn,S,"0356.00, 0357.01, 0357.02, 0363.07, 0364.01, 0...",48507.0,13.34,3636.0,-1.5,26190.0,13.3,16.0,Gujarati (9.1%),09.1% Gujarati,NaN
153,153,Wychwood,OCoT,0116.00,4182.0,0.68,6150.0,-2.0,53613.0,17.1,20.1,Portuguese (2.7%),02.7% Portuguese,NaN
154,154,York Mills,NY,"0273.01, 0273.02, 0274.01, 0274.02",17564.0,7.29,2409.0,2.0,92099.0,10.0,11.8,Korean (4.0%),04.0% Korean,NaN
155,155,York University Heights,NY,"0311.02, 0311.03, 0311.04, 0311.05, 0311.06",26140.0,13.21,1979.0,-1.2,24432.0,15.2,20.4,Italian (6.6%),06.6% Italian,NaN


#### Creatting a new dataframe with required selected columns. 

In [9]:
new_df=df[['Name','FM','Population','Average Income','Second most common language (after English) by name','Second most common language (after English) by percentage']]

In [10]:
new_df.shape

(157, 6)

In [11]:
new_df.head(10)

,Name,FM,Population,Average Income,Second most common language (after English) by name,Second most common language (after English) by percentage
0,Crescent Town,EY,8157.0,23021.0,Bengali (18.1%),18.1% Bengali
1,Governor's Bridge/Bennington Heights,EY,2112.0,129904.0,Polish (1.4%),01.4% Polish
2,Leaside,EY,13876.0,82670.0,Bulgarian (0.4%),00.4% Bulgarian
3,O'Connor–Parkview,EY,17740.0,33517.0,Urdu (3.2%),03.2% Urdu
4,Old East York,EY,52220.0,33172.0,Greek (4.3%),04.3% Greek
5,Thorncliffe Park,EY,17949.0,25340.0,Urdu (21.5%),21.5% Urdu
6,Agincourt,S,44577.0,25750.0,Cantonese (19.3%),19.3% Cantonese
7,Alexandra Park,OCoT,4355.0,19687.0,Cantonese (17.9%),17.9% Cantonese
8,Allenby,OCoT,2513.0,245592.0,Russian (1.4%),01.4% Russian
9,Amesbury,NY,17318.0,27546.0,Spanish (6.1%),06.1% Spanish


#### Renaming the column to understand easily and then sorting it by 2nd_language_percentage

In [12]:
new_df=new_df.rename(columns = {'Average Income':'Avg_income','Second most common language (after English) by name':'2nd_language','Second most common language (after English) by percentage': '2nd_language_percentage'})

In [13]:
new_df.sort_values(by="2nd_language_percentage", ascending=False).head(10)

,Name,FM,Population,Avg_income,2nd_language,2nd_language_percentage
146,Westminster,NY,16386.0,27826.0,Russian (31.4%),31.4% Russian
90,Milliken,S,26272.0,25243.0,Cantonese (26.6%),26.6% Cantonese
21,Branson,NY,8017.0,27156.0,Russian (24.1%),24.1% Russian
85,Little Portugal,OCoT,5013.0,29224.0,Portuguese (23.8%),23.8% Portuguese
125,Steeles,S,24696.0,26660.0,Cantonese (23.7%),23.7% Cantonese
5,Thorncliffe Park,EY,17949.0,25340.0,Urdu (21.5%),21.5% Urdu
23,Brockton,OCoT,9039.0,27260.0,Portuguese (19.9%),19.9% Portuguese
6,Agincourt,S,44577.0,25750.0,Cantonese (19.3%),19.3% Cantonese
142,Wallace Emerson,OCoT,10338.0,25029.0,Portuguese (19.2%),19.2% Portuguese
63,Harwood,Y,3375.0,22136.0,Portuguese (19.0%),19.0% Portuguese


#### Processing data to get the unique language names

In [14]:
name=new_df['2nd_language']


In [15]:
len(name)

157

In [16]:
for ii in range(len(name)):
    a=name[ii]
    print(a)
    ll=int(len(a))
    lf=a.find('(',0, ll)
    print(lf)
    if(lf>-1):
        br=a[0:lf-1]
        name[ii]=br

Bengali (18.1%)
8
Polish (1.4%)
7
Bulgarian (0.4%)
10
Urdu (3.2%)
5
Greek (4.3%)
6
Urdu (21.5%)
5
Cantonese (19.3%)
10
Cantonese (17.9%)
10
Russian (1.4%)
8
Spanish (6.1%)
8
Russian (9.4%)
8
Unspecified Chinese (5.1%)
20
Russian (9.5%)
8
Mandarin (9.6%)
9
Cantonese (8.4%)
10
Cantonese (9.3%)
10
Greek (0.7%)
6
Tamil (3.7%)
6
Unspecified Chinese (0.9%)
20
Ukrainian (3.6%)
10
Greek (4.8%)
6
Russian (24.1%)
8
Cantonese (12.0%)
10
Portuguese (19.9%)
11
Unspecified Chinese (1.6%)
20
Tagalog (2.9%)
8
Portuguese (17.0%)
11
Korean (1.8%)
7
French (1.2%)
7
Portuguese (11.9%)
11
Spanish (1.8%)
8
Filipino (4.3%)
9
Tamil (1.5%)
6
Cantonese (2.3%)
10
Spanish (2.1%)
8
Filipino (1.4%)
9
Portuguese (17.1%)
11
Persian (1.5%)
8
Russian (1.1%)
8
Unspecified Chinese (6.5%)
20
French (2.5%)
7
Unspecified Chinese (3.9%)
20
Unspecified Chinese (9.7%)
20
Tamil (9.5%)
6
Portuguese (13.1%)
11
Italian (11.7%)
8
Portuguese (15.2%)
11
Portuguese (18.9%)
11
Cantonese (4.2%)
10
Tamil (9.7%)
6
Vietnamese (6.9%)
11
Por

<ipython-input-16-2288a2322a87>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  name[ii]=br


name

In [18]:
def get_unique_numbers(numbers):

    list_of_unique_numbers = []

    unique_numbers = set(numbers)

    for number in unique_numbers:
        list_of_unique_numbers.append(number)

    return list_of_unique_numbers

unique_language=get_unique_numbers(name)

print(unique_language)

['Tagalog', 'Bulgarian', 'Japanese', 'Serbian', 'Punjabi', 'Gujarati', 'Mandarin', 'Bengali', 'Filipino', 'None', 'Tamil', 'Hindi', 'Persian', 'Russian', 'Polish', 'Italian', 'French', 'Ukrainian', 'Portuguese', 'Vietnamese', 'Spanish', 'Greek', 'Urdu', 'Unspecified Chinese', 'Cantonese', 'Korean']


#### Here is the total number of unique 2nd language

In [19]:
len(unique_language)

26

In [20]:
new_df['2nd_language']=name


In [21]:
new_df['2nd_language']

0         Bengali
1          Polish
2       Bulgarian
3            Urdu
4           Greek
          ...    
152      Gujarati
153    Portuguese
154        Korean
155       Italian
156        French
Name: 2nd_language, Length: 157, dtype: object

## Methodology: How to find the bengali spoken location

#### After getting the unique language names. I found The language "Begali" location by following codes. There are only 3 locations.

In [22]:
def getIndexes(dfObj, value):
      
    # Empty list
    listOfPos = []
      
    # isin() method will return a dataframe with 
    # boolean values, True at the positions    
    # where element exists
    result = dfObj.isin([value])
      
    # any() method will return 
    # a boolean series
    seriesObj = result.any()
  
    # Get list of column names where 
    # element exists
    columnNames = list(seriesObj[seriesObj == True].index)
     
    # Iterate over the list of columns and
    # extract the row index where element exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
  
        for row in rows:
            listOfPos.append((row, col))
              
    # This list contains a list tuples with 
    # the index of element in the dataframe
    return listOfPos
  
# Calling getIndexes() function to get 
# the index positions of all occurrences
# of 22 in the dataframe
listOfPositions = getIndexes(new_df, 'Bengali')
  
print('Index positions of Bengali in Dataframe : ')
  
# Printing the position
for i in range(len(listOfPositions)):
    print( listOfPositions[i])

Index positions of Bengali in Dataframe : 
(0, '2nd_language')
(97, '2nd_language')
(108, '2nd_language')


In [23]:
len(listOfPositions)

3

## Analysis:
#### Now creating a data frame 'nn' with these 3 begali locations. Use geocode to get the the geograpical coordinate of Bengli locations:

In [24]:
nn = []
ff=[]
aa=[]
pp=[]
gg=[]
location_interrest=[]
for i in range(len(listOfPositions)):
    nn.append(new_df['Name'][listOfPositions[i][0]])
    ff.append(new_df['FM'][listOfPositions[i][0]])
    pp.append(new_df['Population'][listOfPositions[i][0]])
    aa.append(new_df['Avg_income'][listOfPositions[i][0]])
    gg.append(new_df['2nd_language_percentage'][listOfPositions[i][0]])
   

In [25]:
nn=pd.DataFrame(nn)
nn['FM']=pd.DataFrame(ff)
nn['Polpulation']=pd.DataFrame(pp)
nn['Avg_income']=pd.DataFrame(pp)
nn['2nd_language_percentage']=pd.DataFrame(gg)
nn=nn.rename(columns = {0:'Name'})
nn

,Name,FM,Polpulation,Avg_income,2nd_language_percentage
0,Crescent Town,EY,8157.0,8157.0,18.1% Bengali
1,Oakridge,S,13368.0,13368.0,12.6% Bengali
2,Regent Park/Trefann Court,OCoT,10387.0,10387.0,10.5% Bengali


In [27]:
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [28]:
address = nn['Name'][1]
astr2=", Toronto"
address=address+astr2
print(address)
type(address)




Oakridge, Toronto


str

In [29]:
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [30]:
print(latitude)
print(longitude)



43.6971738
-79.2748232


In [31]:
lt=[]
lg=[]
for i in range(len(listOfPositions)):
    address = nn['Name'][i]
    astr2=", Toronto"
    address=address+astr2
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    lt.append(latitude)
    lg.append(longitude)

In [32]:
print('The geograpical coordinate of Bengli are {}, {}.'.format(lt, lg))

The geograpical coordinate of Bengli are [43.695403, 43.6971738, 43.658525], [-79.293099, -79.2748232, -79.3639896].


In [33]:
nn['Latitude']=pd.DataFrame(lt)
nn['Longitude']=pd.DataFrame(lg)
nn

,Name,FM,Polpulation,Avg_income,2nd_language_percentage,Latitude,Longitude
0,Crescent Town,EY,8157.0,8157.0,18.1% Bengali,43.695403,-79.293099
1,Oakridge,S,13368.0,13368.0,12.6% Bengali,43.697174,-79.274823
2,Regent Park/Trefann Court,OCoT,10387.0,10387.0,10.5% Bengali,43.658525,-79.363990


#### Following credentails for Foursquare app

In [52]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Following is the map showing the three selected area

In [35]:
map_toronto= folium.Map(location=[lt[0],lg[0]], zoom_start=10)
for lat, lng, label in zip(nn['Latitude'], nn['Longitude'], nn['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

In [36]:
nn.loc[1, 'Name']

'Oakridge'

### Having the restaurant venues from Foursquare app
#### first checking with only one location. Then with all 3 locations. 

In [37]:
neighborhood_latitude = nn.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = nn.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = nn.loc[1, 'Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Oakridge are 43.6971738, -79.2748232.


In [53]:
# type your answer here
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20180605&ll=43.6971738,-79.2748232&radius=500&limit=100'

In [39]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60add59d358e9a1224ae99c7'},
 'response': {'headerLocation': 'Birch Cliff',
  'headerFullLocation': 'Birch Cliff, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.701673804500004,
    'lng': -79.26861075895759},
   'sw': {'lat': 43.6926737955, 'lng': -79.28103564104241}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c3fb4a1d7fad13a4dfc04da',
       'name': 'Dairy Queen',
       'location': {'address': '10 Danforth Road',
        'lat': 43.694488962802616,
        'lng': -79.27699774503714,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.694488962802616,
          'lng': -79.27699774503714}],
        'distance': 346,
        'postalCode': 'M1L 3W

In [40]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [41]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-41-1d3e4353c370>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Dairy Queen,Ice Cream Shop,43.694489,-79.276998
1,Harvey's,Restaurant,43.694561,-79.275943
2,Premium Sweet,Dessert Shop,43.695566,-79.276021
3,Prince Supermarket,Convenience Store,43.694613,-79.273878
4,Scotia Parkette,Park,43.695304,-79.271826


In [42]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
# type your answer here
begali_venue = getNearbyVenues(names=nn['Name'],
                                   latitudes=nn['Latitude'],
                                   longitudes=nn['Longitude']
                                  )

Crescent Town
Oakridge
Regent Park/Trefann Court


In [45]:
print(begali_venue.shape)
begali_venue.head(32)

(40, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Crescent Town,43.695403,-79.293099,Dentonia Park,43.692809,-79.295533,Park
1,Crescent Town,43.695403,-79.293099,Dentonia Park Golf Course,43.695326,-79.289115,Golf Course
2,Crescent Town,43.695403,-79.293099,Gateway Newstand,43.694477,-79.289194,Convenience Store
3,Crescent Town,43.695403,-79.293099,Victoria Park Subway Station,43.694883,-79.288735,Metro Station
4,Oakridge,43.697174,-79.274823,Dairy Queen,43.694489,-79.276998,Ice Cream Shop
5,Oakridge,43.697174,-79.274823,Harvey's,43.694561,-79.275943,Restaurant
6,Oakridge,43.697174,-79.274823,Premium Sweet,43.695566,-79.276021,Dessert Shop
7,Oakridge,43.697174,-79.274823,Prince Supermarket,43.694613,-79.273878,Convenience Store
8,Oakridge,43.697174,-79.274823,Scotia Parkette,43.695304,-79.271826,Park
9,Oakridge,43.697174,-79.274823,TTC Stop #8555,43.700368,-79.276268,Bus Stop


### Here finding the number of indians restaurants in these 3 locations

In [46]:
listOfPositions = getIndexes(begali_venue[['Venue Category']], 'Indian Restaurant')

In [47]:
len(listOfPositions)

1

In [48]:
print('Index positions of Indian Restaurant in the selected : ')
  
# Printing the position
for i in range(len(listOfPositions)):
    print( listOfPositions[i])
    a=begali_venue['Neighborhood'][listOfPositions[i][0]]
    print('Neighboorhood:',a)
    

Index positions of Indian Restaurant in the selected : 
(21, 'Venue Category')
Neighboorhood: Regent Park/Trefann Court


## Result:

So there is only one indian restaurant in the selected area. So I can start a bangladeshi restaurant in any of three the Bengali laguage spoken areas. I will select the highiest populated area.

In [49]:
final_location=nn.sort_values(by=['Polpulation'],ascending=False)


## Conclusion:
So I will start a new bangladeshi restaurant in following location because this area has the largest Banglaeshi population. And my restaurant will be the first one.

In [50]:
final_location=final_location.head(1)
final_location

,Name,FM,Polpulation,Avg_income,2nd_language_percentage,Latitude,Longitude
1,Oakridge,S,13368.0,13368.0,12.6% Bengali,43.697174,-79.274823


In [51]:
map_toronto= folium.Map(location=[lt[0],lg[0]], zoom_start=10)
for lat, lng, label in zip(final_location['Latitude'], final_location['Longitude'], final_location['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto